In [ ]:
%matplotlib widget

# Session 0

## First X-ray radiograph simulations with ![gVXR](https://gvirtualxray.fpvidal.net/assets/img/Logo.png)

## Author: Franck Vidal

(version 1.0, 22 Sep 2022)

# Main steps

![](img/dragon2.jpg)

There are 6 main steps to simulate an X-ray image:

1. Create a renderer (OpenGL context);
2. Set the X-ray source;
3. Set the Spectrum;
4. Set the Detector;
5. Set the Sample; and
6. Compute the corresponding X-ray image.

# Import packages

In [ ]:
import os
import numpy as np # Who does not use Numpy?

import matplotlib # To plot images
import matplotlib.pyplot as plt # Plotting

font = {'family' : 'serif',
         'size'   : 15
       }
matplotlib.rc('font', **font)

# Uncomment the line below to use LaTeX fonts
# matplotlib.rc('text', usetex=True)

from tifffile import imread, imwrite # Write TIFF files

from gvxrPython3 import gvxr # Simulate X-ray images
from gvxrPython3.utils import saveProjections # Plot the X-ray image in linear, log and power law scales
from gvxrPython3.utils import compareWithGroundTruth # Plot the ground truth, the test image, and the relative error map in %

# Create an OpenGL context

- Create an interactive window (available on Linux, MacOS, and Windows)
- Create a context without a window using EGL (available on Linux and MacOS, but not Windows)
    - use on desktop computers,
    - supercomputers, or 
    - the cloud.

```cpp
//------------------------------------------------------------------------------
/// Create an OpenGL context (the window won't be shown).
/**
 *  @param aWindowID:   the numerical ID of the context to create
 *                      (default value: -1, means that the ID will be
 *                      automatically generated)
 *  @param aRendererMajorVersion: Select the major version of the renderer.
 *                    (default value: 3)
 *  @param aRendererMajorVersion: Select the minor version of the renderer.
 *                    (default value: 2)
 */
//------------------------------------------------------------------------------
void createOpenGLContext(int aWindowID = -1,
		int aRendererMajorVersion = 3,
		int aRendererMinorVersion = 2);
```

```cpp
//------------------------------------------------------------------------------
/// Create an OpenGL context and display it in a window
/**
 *  @param aWindowID:   the numerical ID of the context to create
 *                      (default value: -1, means that the ID will be
 *                      automatically generated)
 *  @param aRenderer: Select the renderer to use, e.g. OpenGL or Vulkan.
 *                    (default value: OPENGL)
 *  @param aRendererMajorVersion: Select the major version of the renderer.
 *                    (default value: 3)
 *  @param aRendererMajorVersion: Select the minor version of the renderer.
 *                    (default value: 2)
 *  @param aVisibilityFlag: flag controling if the window should be visible (1)
 *                          or hidden (0). (default value: 0)
 */
//------------------------------------------------------------------------------
#ifndef __APPLE__
void createWindow(int aWindowID = -1,
		int aVisibilityFlag = 0,
		const std::string& aRenderer = "OPENGL",
		int aRendererMajorVersion = 3,
		int aRendererMinorVersion = 2);
#else // __APPLE__
void createWindow(int aWindowID = -1,
		int aVisibilityFlag = 0,
		const std::string& aRenderer = "OPENGL",
		int aRendererMajorVersion = 2,
		int aRendererMinorVersion = 1);
#endif // __APPLE__
```

In [ ]:
print("Create an OpenGL context")

window_id = 0
opengl_major_version = 4
opengl_minor_version = 5

gvxr.createOpenGLContext(window_id, opengl_major_version, opengl_minor_version);

backend = "OPENGL"
visible = True

# gvxr.createWindow(window_id, visible, backend, opengl_major_version, opengl_minor_version);

visible = False
# gvxr.createWindow(window_id, visible, backend, opengl_major_version, opengl_minor_version);

backend = "EGL"
# visible has no effect with EGL
# gvxr.createWindow(window_id, visible, backend, opengl_major_version, opengl_minor_version);

# Set the X-ray source

- Position
    - x: -40.0 cm,
    - y: 0.0 cm,
    - z: 0.0 cm.
    
```python
gvxr.setSourcePosition(
    x, # float
    x, # float
    z, # float
    unit of length # string, e.g. "cm"
)
```

- Shape:
    - Cone beam: `gvxr.usePointSource()`, or
    - Parallel (e.g. synchrotron): `gvxr.useParallelBeam();`

# Set the spectrum

- monochromatic (0.08 MeV, i.e. 80 keV),
- 1000 photons per ray.

```python
gvxr.setMonoChromatic(
    energy, # float
    unit_of_energy, # string, e.g. "MeV"
    number of photons # float
)
```

# Set the detector:


- Position
    - x: 10.0 cm,
    - y: 0.0 cm,
    - z: 0.0 cm.
    
```python
gvxr.setDetectorPosition(
    x, # float
    x, # float
    z, # float
    unit of length # string, e.g. "cm"
)
```

- Orientation: (unit vector)
    - x: 0.0,
    - y: 0.0,
    - z: -1.0.
    
```python
gvxr.setDetectorUpVector(
    x, # float
    x, # float
    z # float
)
```

- Resolution:
    - width: 640 pixels,
    - height: 320 pixels.

```python
gvxr.setDetectorNumberOfPixels(
    width, # unsigned int
    height # unsigned int
)
```    

- Pixel spacing:
    - width: 0.5 mm,
    - height: 0.5 mm.

```python
gvxr.setDetectorPixelSize(
    width, # float
    height # float
    unit of length # string, e.g. "mm"
)
```    

# Set the sample

- Welsh dragon in a STL file:
    - ID: "Dragon",
    - fname: "input_data/welsh-dragon-small.stl",
    - Unit: mm.
    
```python
gvxr.loadMeshFile(
    ID, # string
    fname, # string
    unit of length # string, e.g. "mm"
)
```    

- Move the sample to the centre of the world:
    - ID: "Dragon",

```python
gvxr.moveToCentre(
    ID # string
)
```    

or

```python
gvxr.moveToCenter(
    ID # string
)
```    

or if you prefer the American spelling

- Material of the sample (ID = "Dragon"):
    - For a chemical element such as iron, you can use the Z number or symbol:
        - `gvxr.setElement("Dragon", 26)`, or
        - `gvxr.setElement("Dragon", "Fe")`
    - For a compound such as water, do not forget to specify the density:
        - `gvxr.setCompound("Dragon", "H2O")`
        - `gvxr.setDensity("Dragon", 1.0, "g/cm3")`
        - `gvxr.setDensity("Dragon", 1.0, "g.cm-3")`
    - For a mixture such as Titanium-Aluminum-Vanadium alloy, do not forget to specify the density:
        - `gvxr.setMixture("Dragon", "Ti90Al6V4")`
        - `gvxr.setMixture("Dragon", [22, 13, 23], [0.9, 0.06, 0.04])`
        - `gvxr.setMixture("Dragon", ["Ti", "Al", "V"], [0.9, 0.06, 0.04]) # Not yet implemented`
        - `gvxr.setDensity("Dragon", 4.43, "g/cm3")`
        - `gvxr.setDensity("Dragon", 4.43, "g.cm-3")`
      
**In this example, we'll use Ti90Al6V4.**

# Compute the corresponding X-ray image

Get a 2D array

```python
xray_image = gvxr.computeXRayImage()
```

or make sure it's a Numpy array in float32 with

```python
xray_image = np.array(gvxr.computeXRayImage()).astype(np.single)
```

# Create the output directory if needed

In [ ]:
if not os.path.exists("output_data"):
    os.mkdir("output_data")

# Save the X-ray image in a TIFF file and store the data using single-precision floating-point numbers.

- using gVXR directly
    - fname = output_data/02-gvxr-save.tif

```python
gvxr.saveLastXRayImage(
    fname # string
)
```

- using the tifffile package
    - fname = output_data/02-tifffile-save.tif

```python
from tifffile import imwrite
imwrite(
    fname, # string
    xray_image # 2D array
)
```

# Display the X-ray image

# using a linear colour scale

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("Image simulated using gVirtualXray\nusing a linear colour scale")
plt.imshow(xray_image, cmap="gray")
plt.colorbar(orientation='vertical');
plt.show()

# using a logarithmic colour scale

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("Image simulated using gVirtualXray\nusing a logarithmic colour scale")
plt.imshow(xray_image, cmap="gray", norm=LogNorm(vmin=xray_image.min(), vmax=xray_image.max()))
plt.colorbar(orientation='vertical');
plt.show()

# using a Power-law colour scale ($\gamma$=0.5)

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("Image simulated using gVirtualXray\nusing a Power-law colour scale ($\gamma=0.5$)")
plt.imshow(xray_image, cmap="gray", norm=PowerNorm(gamma=1./2.))
plt.colorbar(orientation='vertical');
plt.show()

# Display the X-ray image and compare three different lookup tables

In [ ]:
saveProjections("output_data/02-projections-dragon-TiAlV.pdf", xray_image)

# Get some image statistics using Numpy

and compare with the values I got. We should get the same, or at least something comparable.

| What? | Value (in keV) |
|-------|-------|
| Min pixel value: | 0.0056294748 |
| Mean pixel value: | 56.367035 |
| Median pixel value: | 80.0 |
| Stddev pixel value: | 33.96409 |
| Max pixel value: | 80.0 |

In [ ]:
print("Min pixel value:", np.min(xray_image))
print("Mean pixel value:", np.mean(xray_image))
print("Median pixel value:", np.median(xray_image))
print("Stddev pixel value:", np.std(xray_image))
print("Max pixel value:", np.max(xray_image))

# Compare with the ground truth

In [ ]:
ground_truth = imread("input_data/02-dragon-TiAlV-groundtruth.tif")
compareWithGroundTruth(ground_truth, xray_image)

# Update the visualisation window

In [ ]:
gvxr.displayScene()

# Change the sample's colour

By default the object is white, which is not always pretty. Let's change it to purple.

In [ ]:
red = 102 / 255
green = 51 / 255
blue = 153 / 255
gvxr.setColour("Dragon", red, green, blue, 1.0)

# This image can be used in a research paper to illustrate the simulation environment, in which case you may want to change the background colour to white with:

In [ ]:
gvxr.setWindowBackGroundColour(1.0, 1.0, 1.0)

# Update the visualisation window

In [ ]:
gvxr.displayScene()

# Take the screenshot and save it in a file

In [ ]:
screenshot = gvxr.takeScreenshot()
plt.imsave("output_data/02-screenshot.png", np.array(screenshot))

# or display it using Matplotlib

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(screenshot)
plt.title("Screenshot of the X-ray simulation environment")
plt.axis('off');

# Interactive visualisation

The user can rotate the 3D scene and zoom-in and -out in the visualisation window.

- Keys are:
    - Q/Escape: to quit the event loop (does not close the window)
    - B: display/hide the X-ray beam
    - W: display the polygon meshes in solid or wireframe
    - N: display the X-ray image in negative or positive
    - H: display/hide the X-ray detector
- Mouse interactions:
    - Zoom in/out: mouse wheel
    - Rotation: Right mouse button down + move cursor
    
**Note: this function has no effect on supercomputers and on the cloud, but will work on a desktop or laptop computer.**

In [ ]:
gvxr.renderLoop()

# All done

In [ ]:
gvxr.terminate()